# <center>*IMPACT PROJECT - GESTAMP*</center> 
## <center>Defect Detection using Machine Learning</center> 
### <center>Cleaning Data</center>
_____

<img 
    src="https://www.gestamp.com/getattachment/c8d61c0f-e752-4156-8002-97e21ab43a3f/Imag2-2" width="2400" height="1000" align="center"/>

___

## <center>*Table of Contents*</center>
1. [Data Cleaning](#1)
    * 1.1. [Changing CSV file separator and creating Datetime objects](#1.1)
    * 1.2. [Cleaning the Defects CSV file](#1.2)
2. [Merging](#2)
    * 2.1. [Merging per Press](#2.1)
    * 2.2. [Merging for All](#2.2)

<a id='1'></a>

## <center>*1. Data Cleaning*</center>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import glob
import os

<a id='1.1'></a>

### <center>*1.1. Changing CSV file separator and creating Datetime objects*</center>

In [5]:
# Zip file path
zip_file_path = "MPO.01.zip"

# Pattern to match CSV files
file_pattern = "*.csv"

In [6]:
# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    # Get the file names matching the pattern for P03
    file_names = [file for file in zip_file.namelist() if file.endswith('.csv')]

    for file_name in file_names:
        # Extract the CSV file
        zip_file.extract(file_name)

        # Read the CSV file
        df = pd.read_csv(file_name, delimiter=";")

        # Keep only document name
        file_name = file_name.split("/")[-1]
        print(file_name)

        # Loop to replace "," in y-variable column and convert into float
        for column_name in df.columns:
            if df[column_name].dtype == object and df[column_name].str.contains(",").any():
                df[column_name] = df[column_name].str.replace(",", ".").astype(float)
        
        # Convert time columns into datetime
        for col_name in df.columns:
            if ("time" in col_name.lower()) & ("time" not in file_name.lower()):
                print(col_name)
                df[col_name] = pd.to_datetime(df[col_name].str.replace(" CET", ""))
            elif (col_name=="time") | (col_name=="time.value"):                         #exception for the one file that includes "Time" in the file name
                print(col_name)
                df[col_name] = pd.to_datetime(df[col_name].str.replace(" CET", ""))

        # Filter columns with float data type
        float_columns = df.select_dtypes(include=[float])

        #save the cleaned csv
        if file_name[:2] == "P0":
            df.to_csv(f"datasets/clean_csv/{file_name[:3]}/{file_name}")
        else:
            print("other")
            df.to_csv(f"datasets/clean_csv/{file_name}")

Defects.csv
time
other
P01-AmbientHumidity-values.csv
time
time.value
P01-AmbientTemperature-values.csv
time
time.value
P01-ClutchBrakeTemperatureUnitOil-values.csv
time
time.value
P01-ClutchBrakeWaterTemperature-values.csv
time
time.value
P01-CushionOilDegradation-values.csv
time
time.value
P01-CushionPumpMaxPower1-values.csv
time
time.value
P01-CushionPumpMaxPower2-values.csv
time
time.value
P01-CushionPumpMaxPower3-values.csv
time
time.value
P01-CushionPumpMeanPower1-values.csv
time
time.value
P01-CushionPumpMeanPower2-values.csv
time
time.value
P01-CushionPumpMeanPower3-values.csv
time
time.value
P01-CushionTemperatureUnitOil-values.csv
time
time.value
P01-CushionWaterFlow-values.csv
time
time.value
P01-CushionWaterTemperature-values.csv
time
time.value
P01-Cylinder1MaxForce-values.csv
time
time.value
P01-Cylinder1MaxParalelismErrorInBDC-values.csv
time
time.value
P01-Cylinder1MaxParallelismInCycle-values.csv
time
time.value
P01-Cylinder2MaxForce-values.csv
time
time.value
P01-Cyli

<a id='1.2'></a>

### <center>*1.2. Cleaning the Defects CSV file*</center>

In [ ]:
defects = pd.read_csv("datasets/clean_csv/defects.csv")

defects = defects.drop(columns="Unnamed: 0")

defects["Defect"] = defects["Defect"].str.split(r'\(|\)').str.get(1)
defects["Defect"] = defects["Defect"].fillna("Other")

defects.to_csv("datasets/merged_csv/Defects.csv")

<a id='2'></a>

## <center>*2. Data Merging*</center>

<a id='2.1'></a>

### <center> 2.1. Merging per Press </center>

In [ ]:
def get_df_P0(num):
    # Directory where the CSV files are located
    directory = f"datasets/clean_csv/P0{num}"

    # Pattern to match CSV files
    file_pattern = "*.csv"

    # Get the file names matching the pattern for P03
    file_names = glob.glob(directory + "/" + file_pattern)


    value_df = pd.DataFrame()
    other_df = pd.DataFrame()

    for indx, file_name in enumerate(file_names):
        # Read the CSV file
        df = pd.read_csv(file_name)
        df = df.drop(columns="Unnamed: 0")

        #keep only document name
        file_name=file_name.split("\\")[1]
        print(file_name)

        if "value" in file_name:
            if indx == 0:
                value_df = df
                continue

            if "TraceabilityCode" in df.columns:                #check for traceability code in the columns
                merge_columns = ["TraceabilityCode", "time", "time.value"]
                
                value_df = pd.merge(value_df,df,how="outer",on=merge_columns,suffixes=("",f"_{indx}"))
        else:
            if other_df.empty:
                other_df = df
                continue
            if "TraceabilityCode" in df.columns:                #check for traceability code in the columns
                merge_columns = ["TraceabilityCode", "time"]
                
                other_df = pd.merge(other_df,df,how="outer",on=merge_columns,suffixes=("",f"_{indx}"))
    
    value_df = value_df.drop(columns="time.value")
    value_df = value_df.groupby(["TraceabilityCode", "time"]).mean(numeric_only=True).reset_index() #grouping all ["TraceabilityCode", "time"] duplicates and keep the avg

    return value_df, other_df

In [ ]:
def merge_DieReference(value, other):
    merged_df = pd.merge(value, other[["TraceabilityCode","DieReference"]], on="TraceabilityCode")
    return merged_df

In [ ]:
def df_to_csv(df, id, type=0):
    print(f"{id}_{type}")
    if type == 0:
        df.to_csv(f"datasets/merged_csv/{id}.csv")
    else:
        df.to_csv(f"datasets/merged_csv/P0{id}/P0{id}_{type}.csv")

In [ ]:
values = []
others = []
for i in range (1,7):                                   #looping through every P0_ section of files, excluding P01
    df_value, df_other = get_df_P0(i)                   #merging every p0_ dataset together into one df, separated by P0_

    df_value = merge_DieReference(df_value, df_other)   #merge DieReference column to df_value
    
    values.append(df_value)                             #saving the sizes of the Dataframes
    others.append(df_other)

In [ ]:
for i in range(0,len(values)):
    df_to_csv(values[i], i+1, "value")                #saving value dfs to csv
    if i != 0:
        df_to_csv(others[i], i+1, "other")            #saving other dfs to csv

#### <center> Run the fill_nans.ipynb file </center>

<a id='2.2'></a>

### <center> 2.2. Merging for All </center>

<center> Load in all the files </center>

In [ ]:
zip_file_path = 'datasets/filled_csv/P01_filled.zip'

# Name of the CSV file inside the zip
csv_file_name = 'data.csv'

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    # Read the CSV file from the zip
    with zip_file.open(csv_file_name) as csv_file:
        # Read the CSV file as a DataFrame
        df = pd.read_csv(csv_file)

In [ ]:
# Directory where the CSV files are located
directory = f"datasets/filled_csv"

# Pattern to match CSV files
file_pattern = "*.csv"

# Get the file names matching the pattern for P03
file_names = glob.glob(directory + "/" + file_pattern)

filled_dfs = [df]

for indx, file_name in enumerate(file_names):
    # Read the CSV file
    df = pd.read_csv(file_name)
    df = df.drop(columns=["Unnamed: 0","Unnamed: 0.1"])
    filled_dfs.append(df)

In [ ]:
defects = pd.read_csv("datasets/merged_csv/Defects.csv")

<center> Merging </center>

In [ ]:
global_csv = filled_dfs[0]
for i in range(1,len(filled_dfs)):
    global_csv = pd.merge(global_csv,filled_dfs[i],how="outer",on=["TraceabilityCode"],suffixes=("",f"_0{i+1}"))

In [ ]:
global_csv = pd.merge(global_csv,defects[["TraceabilityCode","Defect"]],how="left",on=["TraceabilityCode"])
global_csv = global_csv.drop(columns=["Unnamed: 0","Unnamed: 0.1"])

In [ ]:
global_csv.head()

In [ ]:
global_csv.to_csv("datasets/filled_csv/global.csv")

In [ ]:
def compress_csv_to_zip(csv_file_path, zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(csv_file_path, arcname='data.csv')
    
    os.remove(csv_file_path)

# Specify the paths to your CSV and the desired output ZIP file
csv_file_path = 'datasets/filled_csv/global.csv'
zip_file_path = 'datasets/filled_csv/global.zip'

# Call the function to compress the CSV into a ZIP archive
compress_csv_to_zip(csv_file_path, zip_file_path)